In [4]:
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import json 
import ast

In [5]:
movies=pd.read_csv('DataSets/movies_metadata.csv')

In [6]:
credits=pd.read_csv('DataSets/credits.csv')

In [7]:
credits=credits.drop(credits.columns[2],axis=1)

In [8]:
Data=movies.join(credits)

In [9]:
cols=['id','genres','original_language','original_title','overview','cast','crew']

In [10]:
Final_data=Data[cols]

In [11]:
Final_data.dropna()

,id,genres,original_language,original_title,overview,cast,crew
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,Jumanji,When siblings Judy and Peter discover an encha...,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",en,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",en,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...
45458,439050,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",fa,رگ خواب,Rising and falling between a man and woman.,[],"[{'credit_id': '52fe4a74c3a368484e1542e9', 'de..."
45459,111109,"[{'id': 18, 'name': 'Drama'}]",tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,"[{'cast_id': 1, 'character': 'Roo (voice)', 'c...","[{'credit_id': '52fe46249251416c7506e969', 'de..."
45460,67758,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",en,Betrayal,"When one of her hits goes wrong, a professiona...","[{'cast_id': 3, 'character': 'All the members ...","[{'credit_id': '52fe478dc3a36847f813bd6b', 'de..."
45461,227506,[],en,Satana likuyushchiy,"In a small town live two brothers, one a minis...","[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '52fe4a71c3a36847f81ce35b', 'de..."


In [12]:
crew_list=Final_data['crew']
cast_list=Final_data['cast']
movie_list=Final_data['genres']


In [13]:
def ConvertToDict(ColumnsList,Resultant,Size):
    for i in range (0,Size,1):
        new_list=[]
        dict1=[]
        found=False
        for j in range(0,len(ColumnsList[i]),1):
            if (ColumnsList[i][j]=='{'):
                found=True
            elif ColumnsList[i][j]=='}':
                if ColumnsList[i][j]:
                    new_list+=ColumnsList[i][j]
                found=False
                list1=''.join(new_list)
                dict1.append(ast.literal_eval(list1))
                new_list=[]

            if found==True:
                if ColumnsList[i][j]:
                    new_list+=ColumnsList[i][j]
        Resultant.append(dict1)
    return Resultant


In [14]:
FinalCrewList=[]
FinalCrewList=ConvertToDict(crew_list,FinalCrewList,100)

In [15]:
FinalMovieList=[]
FinalMovieList=ConvertToDict(movie_list,FinalMovieList,100)

In [16]:
FinalCastList=[]
FinalCastList=ConvertToDict(cast_list,FinalCastList,100)

In [17]:
def EctractMeaningful(List,Resultant,Size):
    index=0
    empty=True
    for i in range(0,Size,1):
        for j in range(0,len(List[i]),1):
            found = False
            temp=List[i][j]
            if temp!='':
                if(empty!=True):
                    for k in range(0,len(Resultant),1):
                        if(List[i][j]==Resultant[k]):
                            found=True
                    if found==False:
                        Resultant.insert(index, temp)
                        index+=1
                else:
                    empty=False
                    Resultant.insert(index, temp)
                    index+=1
    return Resultant

In [18]:
def DictToWords(List,Resultant,String,Size):
    feild=[]
    for i in range (0,len(List)):
        temp_list=[]
        for j in range (0,len(List[i])):
            temp_list.append(List[i][j][String])
        feild.append(temp_list)
    Resultant=EctractMeaningful(feild,Resultant,len(feild))
    return Resultant
Genres_=[]
Genres_=DictToWords(FinalMovieList,Genres_,'name',len(FinalMovieList))
jobs=[]
jobs=DictToWords(FinalCrewList,jobs,'job',len(FinalCrewList))

In [19]:
Genre_vector=np.zeros((len(FinalMovieList),len(jobs)), dtype=int)

In [20]:
def ToVect(List,Tags,String,Size):
    Resultant=np.zeros((len(FinalMovieList),len(Tags)), dtype=int)
    for i in range (0,Size):
        for j in range(0,len(List[i])):
            index=-1
            for k in range(0,len(Tags)):
                if List[i][j][String]==Tags[k]:
                    index=k
                    break
            if (index!=-1):
                Resultant[i][index]=1
    return Resultant


In [21]:
Genre=ToVect(FinalMovieList,Genres_,'name',len(FinalMovieList))

In [22]:
GenresDF=pd.DataFrame(Genre,columns=Genres_)

In [23]:
temp=[]
Job_positions = {}
for i in jobs:
    Job_positions[i] = []
for i in range(0,len(FinalCrewList),1):
    for j in range(0,len(FinalCrewList[i]),1):
        index=-1
        for k in range(0,len(jobs)):
            if FinalCrewList[i][j]['job']==jobs[k]:
                index=k
        if (index!=-1):
            Job_positions[jobs[index]].append(FinalCrewList[i][j]['name']) 

In [24]:
for i in range(0,len(jobs)):  
        Job_positions[jobs[i]]=list(set(Job_positions[jobs[i]]))

In [25]:
def ToVectUpdate(List,Tags,String,Type,ParentTag,Size):
    Resultant=np.zeros((len(FinalMovieList),len(Tags)), dtype=int)
    for i in range (0,Size):
        for j in range(0,len(List[i])):
            index=-1
            found=False
            if List[i][j][Type]==ParentTag:
                    found=True
            if found==True:
                for k in range(0,len(Tags)): 
                    if List[i][j][String]==Tags[k]:
                        index=k
                        break
            if (index!=-1):
                Resultant[i][index]=1
    return Resultant

In [41]:
import csv
Allcast= pd.DataFrame()
temp=np.empty([0,0],dtype=np.int8)
temp2=np.empty([0])
sum_=0
for i in range(0,len(jobs)):
    Vectors=ToVectUpdate(FinalCrewList,Job_positions[jobs[i]],'name','job',jobs[i],len(FinalCrewList))
  #  Vectors = pd.DataFrame(Vectors,columns=Job_positions[jobs[i]])
  #  Vectors.to_csv('\simple\Vector '+str(i)+'.csv')
    Vectors=Vectors.tolist()
    if i==0:
        
        temp=Vectors
        temp2=Job_positions[jobs[i]]
    else:
        
        temp=np.concatenate((temp,Vectors), axis=1, out=None)
        temp2=np.concatenate((temp2,Job_positions[jobs[i]]))
(Vectors)
print (len(jobs))
#Vectors = pd.DataFrame(temp,columns=temp2)
#Vectors

211


In [27]:
Vectors=ToVectUpdate(FinalCrewList,Job_positions[jobs[2]],'name','job',jobs[2],len(FinalCrewList))

In [27]:
Vectors2=ToVectUpdate(FinalCrewList,Job_positions[jobs[1]],'name','job',jobs[1],len(FinalCrewList))

In [146]:
Actors=[]
Actors=DictToWords(FinalCastList,Actors,'name',len(FinalCastList))

In [147]:
actor_vector=ToVect(FinalCastList,Actors,'name',len(FinalCastList))

In [148]:
AllActors = pd.DataFrame(actor_vector,columns=Actors)

In [149]:
Final_DataFrame_vector=pd.DataFrame()

In [150]:
Final_DataFrame_vector=pd.concat([GenresDF,Allcast],axis=1)

In [151]:
Final_DataFrame_vector=pd.concat([Final_DataFrame_vector,AllActors],axis=1)

In [152]:
Final_DataFrame_vector=Final_DataFrame_vector.join(Data['id'])

In [159]:
Final_DataFrame_vector.to_csv('Atributes_vectors.csv',index=False)

In [23]:
print (len(FinalCastList))

45463


In [29]:
len(jobs)

211

In [87]:
sum_=0
for i in range(0,len(jobs)):
    for j in range(0,len(jobs[i])):
        print (sum_,jobs[j])
        sum_+=1
print (sum_)

0 Director
1 Screenplay
2 Producer
3 Executive Producer
4 Editor
5 Art Direction
6 Foley Editor
7 Animation
8 Director
9 Screenplay
10 Producer
11 Executive Producer
12 Editor
13 Art Direction
14 Foley Editor
15 Animation
16 ADR Editor
17 Orchestrator
18 Director
19 Screenplay
20 Producer
21 Executive Producer
22 Editor
23 Art Direction
24 Foley Editor
25 Animation
26 Director
27 Screenplay
28 Producer
29 Executive Producer
30 Editor
31 Art Direction
32 Foley Editor
33 Animation
34 ADR Editor
35 Orchestrator
36 Color Timer
37 CG Painter
38 Original Story
39 Post Production Supervisor
40 Sculptor
41 Animation Director
42 Music
43 Layout
44 Director
45 Screenplay
46 Producer
47 Executive Producer
48 Editor
49 Art Direction
50 Director
51 Screenplay
52 Producer
53 Executive Producer
54 Editor
55 Art Direction
56 Foley Editor
57 Animation
58 ADR Editor
59 Orchestrator
60 Color Timer
61 CG Painter
62 Original Story
63 Director
64 Screenplay
65 Producer
66 Executive Producer
67 Editor
68 Art

963 Director
964 Screenplay
965 Producer
966 Executive Producer
967 Editor
968 Art Direction
969 Foley Editor
970 Animation
971 ADR Editor
972 Orchestrator
973 Color Timer
974 CG Painter
975 Original Story
976 Post Production Supervisor
977 Sculptor
978 Animation Director
979 Music
980 Layout
981 Music Editor
982 Negative Cutter
983 Title Designer
984 Supervising Technical Director
985 Director
986 Screenplay
987 Producer
988 Executive Producer
989 Editor
990 Art Direction
991 Foley Editor
992 Animation
993 ADR Editor
994 Orchestrator
995 Color Timer
996 CG Painter
997 Original Story
998 Post Production Supervisor
999 Sculptor
1000 Animation Director
1001 Music
1002 Layout
1003 Music Editor
1004 Negative Cutter
1005 Title Designer
1006 Supervising Technical Director
1007 Songs
1008 Supervising Animator
1009 Director
1010 Screenplay
1011 Producer
1012 Executive Producer
1013 Editor
1014 Art Direction
1015 Foley Editor
1016 Animation
1017 ADR Editor
1018 Orchestrator
1019 Color Timer
102

2046 Editor
2047 Art Direction
2048 Foley Editor
2049 Animation
2050 ADR Editor
2051 Orchestrator
2052 Color Timer
2053 CG Painter
2054 Original Story
2055 Post Production Supervisor
2056 Sculptor
2057 Animation Director
2058 Music
2059 Layout
2060 Music Editor
2061 Negative Cutter
2062 Title Designer
2063 Director
2064 Screenplay
2065 Producer
2066 Executive Producer
2067 Editor
2068 Art Direction
2069 Foley Editor
2070 Animation
2071 ADR Editor
2072 Orchestrator
2073 Color Timer
2074 CG Painter
2075 Original Story
2076 Post Production Supervisor
2077 Sculptor
2078 Director
2079 Screenplay
2080 Producer
2081 Executive Producer
2082 Editor
2083 Art Direction
2084 Foley Editor
2085 Animation
2086 ADR Editor
2087 Orchestrator
2088 Color Timer
2089 CG Painter
2090 Original Story
2091 Post Production Supervisor
2092 Sculptor
2093 Animation Director
2094 Music
2095 Layout
2096 Music Editor
2097 Director
2098 Screenplay
2099 Producer
2100 Executive Producer
2101 Editor
2102 Art Direction
210

3248 Screenplay
3249 Producer
3250 Executive Producer
3251 Editor
3252 Art Direction
3253 Foley Editor
3254 Animation
3255 ADR Editor
3256 Orchestrator
3257 Color Timer
3258 CG Painter
3259 Original Story
3260 Director
3261 Screenplay
3262 Producer
3263 Executive Producer
3264 Editor
3265 Art Direction
3266 Foley Editor
3267 Animation
3268 ADR Editor
3269 Orchestrator
3270 Color Timer
3271 CG Painter
3272 Original Story
3273 Post Production Supervisor
3274 Sculptor
3275 Animation Director
3276 Music
3277 Layout
3278 Music Editor
3279 Negative Cutter
3280 Title Designer
3281 Supervising Technical Director
3282 Songs
3283 Director
3284 Screenplay
3285 Producer
3286 Executive Producer
3287 Editor
3288 Art Direction
3289 Foley Editor
3290 Animation
3291 Director
3292 Screenplay
3293 Producer
3294 Executive Producer
3295 Editor
3296 Art Direction
3297 Foley Editor
3298 Animation
3299 ADR Editor
3300 Orchestrator
3301 Color Timer
3302 CG Painter
3303 Original Story
3304 Post Production Super

In [ ]:
  if i==0:
        temp=Vectors
        temp2=Job_positions[jobs[i]]
    else:
        temp=np.concatenate((temp,Vectors), axis=1, out=None)
        temp2=np.concatenate((temp2,Job_positions[jobs[i]]))
Vectors = pd.DataFrame(temp,columns=temp2)
Vectors